<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Description of the columns:
* name — the name of the country.
* area— the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's waterarea in square kilometers.
* population — the country's population.
* population_growth— the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.

In [5]:
%%sql
SELECT COUNT(*)
FROM facts;

 * sqlite:///factbook.db
Done.


COUNT(*)
261


Hold on - total entry count is 261 but a quick google search lets us know that there are a total of 195 countries in the world. We can assume that other entities are included.

Let's first take a look at the MIN and MAX each column

In [7]:
%%sql
SELECT MIN(area), MAX(area),
       MIN(area_land), MAX(area_land),
       MIN(area_water), MAX(area_water),
       MIN(population), MAX(population),
       MIN(population_growth), MAX(population_growth), 
       MIN(birth_rate), MAX(birth_rate),
       MIN(death_rate), MAX(death_rate),
       MIN(migration_rate), MAX(migration_rate) 
FROM facts;
    

 * sqlite:///factbook.db
Done.


MIN(area),MAX(area),MIN(area_land),MAX(area_land),MIN(area_water),MAX(area_water),MIN(population),MAX(population),MIN(population_growth),MAX(population_growth),MIN(birth_rate),MAX(birth_rate),MIN(death_rate),MAX(death_rate),MIN(migration_rate),MAX(migration_rate)
0,17098242,0,16377742,0,891163,0,7256490011,0.0,4.02,6.65,45.45,1.53,14.89,0.0,22.39


This statistical summary provides with some things to look into:
* there at least one country with a population of 0
* there a country with a max population of over 7 billion people. This doesn't seem right as it should equate to the world population (not country).
* there's at least one country with an area of 0
* I'd like to take a closer look at the country(ies) that have a population growth of 0. 
* As well as take a look at the counties with the highest migration rates.

Let's use subqueries to first zoom in on just the countries with the min and max populations.

In [20]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MIN(population)
                    FROM facts);

 * sqlite:///factbook.db
Done.


name
Antarctica


Quick note: I forgot to add the column population after WHERE. I only added the subquery.

In [22]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts);

 * sqlite:///factbook.db
Done.


name
World


Our table contains a row for the world which explains the population of 7.2 billion people. There is also a row for the Antartica and the population of 0 that is explained on the CIA Factbook description for Antartica. 

Now we can recalculate the summary statistics and exclude the row for the world.

In [31]:
%%sql
SELECT MIN(area), MAX(area),
       MIN(area_land), MAX(area_land),
       MIN(area_water), MAX(area_water),
       MIN(population), MAX(population),
       MIN(population_growth), MAX(population_growth), 
       MIN(birth_rate), MAX(birth_rate),
       MIN(death_rate), MAX(death_rate),
       MIN(migration_rate), MAX(migration_rate) 
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


MIN(area),MAX(area),MIN(area_land),MAX(area_land),MIN(area_water),MAX(area_water),MIN(population),MAX(population),MIN(population_growth),MAX(population_growth),MIN(birth_rate),MAX(birth_rate),MIN(death_rate),MAX(death_rate),MIN(migration_rate),MAX(migration_rate)
0,17098242,0,16377742,0,891163,0,1367485388,0.0,4.02,6.65,45.45,1.53,14.89,0.0,22.39


In [35]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts
                    WHERE name != 'World');

 * sqlite:///factbook.db
Done.


name
China


Quick note: I was trying to add and an AND operator after the subquery instead of adding the where name is notworld to the subquery itself. 

In [42]:
%%sql
SELECT ROUND(AVG(population),2) as avg_pop, 
       ROUND(AVG(area),2) as avg_area
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.57,555093.55


The averages were rounded out to two decimals and columns renames for ease.

The average population is a little over 32 million and the average area is 555 thousand square kilometers.

We'll used the information about to find the countries that are densely populated. In particular, we want to find the countries that have:
* above-average values for population, and
* those that have below-average values for area

In [53]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts
                    WHERE name != 'World')
AND area < (SELECT AVG(area)
            FROM facts
            WHERE name != 'World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Other things to explore:\
Which country has the most people? Which country has the highest growth rate?
Which countries have the highest ratios of water to land? Which countries have more water than land?
Which countries will add the most people to their populations next year?
Which countries have a higher death rate than birth rate?
Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?
